In [1]:
import os
os.chdir(r'C:\Users\Pia\OneDrive\Data Science\experiment_design\ex_2\CoE_dataset')
import xml.etree.ElementTree as et

from metadata_ratings_helper_functions import load_data, get_dummies

from random import seed, sample, choice
import numpy as np
import pandas as pd
pd.options.display.max_columns = 34
pd.options.display.max_rows = 100

In [2]:
df_dev = load_data('Dev')
labels = pd.read_excel(r'Dev_Set\dev_set_groundtruth_and_trailers.xls', index_col=0).goodforairplane
df_dev = df_dev.merge(labels, left_index=True, right_index=True)

df_test = load_data('Test')
labels = pd.read_csv(r'Test_Set\CoeTestLabels.csv', index_col=0).goodforairplanes
df_test = df_test.merge(labels, left_index=True, right_index=True)
df_test.rename(columns={'goodforairplanes': 'goodforairplane'}, inplace=True)

df_dev_test = df_dev.append(df_test)
df_dev_test = get_dummies(df_dev_test)
df_dev_test.index = df_dev_test.index.str.replace('\'', '')


df_dev = df_dev_test.iloc[:86]
# df_dev.to_csv('metadata_ratings_dev.csv')
print(df_dev.head(1))

df_test = df_dev_test.iloc[86:]
# df_test.to_csv('metadata_ratings_test.csv')
df_test

                   year  runtime  metascore  imdbRating  tomatoUserRating  \
American Gangster  2007    157.0       76.0         7.8               4.0   

                   goodforairplane  country_Australia  country_Bahamas  \
American Gangster                1                  0                0   

                   country_Belgium  country_Bosnia and Herzegovina  \
American Gangster                0                               0   

                   country_Bulgaria  country_Canada  country_China  \
American Gangster                 0               0              0   

                   country_Czech Republic  country_Denmark  country_Egypt  \
American Gangster                       0                0              0   

                   country_France  ...  language_Portuguese  language_Russian  \
American Gangster               0  ...                    0                 0   

                   language_Scottish Gaelic  language_Serbian  \
American Gangster              

,year,runtime,metascore,imdbRating,tomatoUserRating,goodforairplane,country_Australia,country_Bahamas,country_Belgium,country_Bosnia and Herzegovina,country_Bulgaria,country_Canada,country_China,country_Czech Republic,country_Denmark,country_Egypt,country_France,...,language_Portuguese,language_Russian,language_Scottish Gaelic,language_Serbian,language_Sign Languages,language_Spanish,language_Swahili,language_Turkish,language_Urdu,language_Vietnamese,language_Yiddish,rated_G,rated_N/A,rated_PG,rated_PG-13,rated_R,rated_TV-MA
10.000 Km,2014,99.0,NaN,6.8,3.7,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
12 Years a Slave,2013,134.0,97.0,8.1,NaN,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
21 Jump Street,2012,109.0,69.0,7.2,4.0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2 States,2014,149.0,NaN,7.1,3.6,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
Aanmodderfakker,2014,100.0,NaN,6.9,NaN,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wassup Rockers,2005,111.0,56.0,6.0,3.4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
Watchmen,2009,162.0,56.0,7.6,3.4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
Whiplash,2014,107.0,88.0,8.6,4.5,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
Wish I Was Here,2014,106.0,43.0,6.7,3.5,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [3]:
ratings = ['metascore', 'imdbRating', 'tomatoUserRating']

metadata_dev = df_dev[df_dev.columns.difference(ratings)]
# metadata_dev.to_csv('metadata_dev.csv')
print(metadata_dev.head(1))

metadata_test = df_test[df_test.columns.difference(ratings)]
# metadata_test.to_csv('metadata_test.csv')
metadata_test

                   country_Australia  country_Bahamas  country_Belgium  \
American Gangster                  0                0                0   

                   country_Bosnia and Herzegovina  country_Bulgaria  \
American Gangster                               0                 0   

                   country_Canada  country_China  country_Czech Republic  \
American Gangster               0              0                       0   

                   country_Denmark  country_Egypt  country_France  \
American Gangster                0              0               0   

                   country_Germany  country_Hong Kong  country_Iceland  \
American Gangster                0                  0                0   

                   country_India  country_Ireland  country_Italy  ...  \
American Gangster              0                0              0  ...   

                   language_Scottish Gaelic  language_Serbian  \
American Gangster                         0            

,country_Australia,country_Bahamas,country_Belgium,country_Bosnia and Herzegovina,country_Bulgaria,country_Canada,country_China,country_Czech Republic,country_Denmark,country_Egypt,country_France,country_Germany,country_Hong Kong,country_Iceland,country_India,country_Ireland,country_Italy,...,language_Scottish Gaelic,language_Serbian,language_Sign Languages,language_Spanish,language_Swahili,language_Turkish,language_Urdu,language_Vietnamese,language_Yiddish,rated_G,rated_N/A,rated_PG,rated_PG-13,rated_R,rated_TV-MA,runtime,year
10.000 Km,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,99.0,2014
12 Years a Slave,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,134.0,2013
21 Jump Street,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,109.0,2012
2 States,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,149.0,2014
Aanmodderfakker,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,100.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wassup Rockers,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,111.0,2005
Watchmen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,162.0,2009
Whiplash,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,107.0,2014
Wish I Was Here,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,106.0,2014


In [4]:
ratings_goodforairplane = ratings + ['goodforairplane']

ratings_dev = df_dev[ratings_goodforairplane]
# ratings_dev.to_csv('ratings_dev.csv')
print(ratings_dev.head(1))

ratings_test = df_test[ratings_goodforairplane]
# ratings_test.to_csv('ratings_test.csv')
ratings_test

                   metascore  imdbRating  tomatoUserRating  goodforairplane
American Gangster       76.0         7.8               4.0                1


,metascore,imdbRating,tomatoUserRating,goodforairplane
10.000 Km,NaN,6.8,3.7,1
12 Years a Slave,97.0,8.1,NaN,1
21 Jump Street,69.0,7.2,4.0,1
2 States,NaN,7.1,3.6,1
Aanmodderfakker,NaN,6.9,NaN,1
...,...,...,...,...
Wassup Rockers,56.0,6.0,3.4,0
Watchmen,56.0,7.6,3.4,0
Whiplash,88.0,8.6,4.5,0
Wish I Was Here,43.0,6.7,3.5,1


In [2]:
#def load_data(data_set):
#
#    # get the columnnames:
#    xtree = et.parse(r'Dev_Set\XML\A_Fish_Called_Wanda.xml')
#    xroot = xtree.getroot()
#    columns = list(xroot.find('movie').keys())
#    df = pd.DataFrame(columns = columns)
#
#    for movie_name in os.listdir(data_set + '_Set/XML'):  
#        xtree = et.parse(data_set + '_Set/XML/' + movie_name)
#        xroot = xtree.getroot()
#        instance = []
#        for element in columns:
#            if xroot[0] is not None: # and element != 'goodforairplane': # feature "good for airplane" is not filled in 
#                instance.append(xroot.find('movie').get(element))
#            else:
#                pass
#        if len(instance) < len(columns):
#            instance.append(movie_name[:-3])
#        my_series = pd.Series(instance, index = columns)
#        df = df.append(my_series, ignore_index=True)
#
#    df.set_index('title', inplace=True)
#
#    df = df.loc[:, ['language', 'year', 'genre', 'country', 'runtime', 'rated', 'metascore', 'imdbRating', 'tomatoUserRating']]
#
#    # make the feature 'runtime' numeric:
#    df.runtime = df.runtime.apply(lambda x: x[:-4])
#    df.runtime = pd.to_numeric(df.runtime, errors='coerce')
#
#    # bring the entries of 'rated' which were not filled out into a unique shape:
#    df.rated.replace(['NOT RATED', 'UNRATED'], 'N/A', inplace=True)
#
#    for feature in ['year', 'metascore', 'imdbRating', 'tomatoUserRating']:
#        df.loc[:, feature] = pd.to_numeric(df.loc[:, feature] , errors='coerce')
#
#    return df
#
#
#def get_dummies(df, selected_features=['language', 'genre', 'country', 'rated']):
#    
#    df_tmp = df.copy()
#    for feature in set(selected_features).intersection(['language', 'genre', 'country', 'rated']):  # we only get dummy 
#                                                                                                     # variables for categorical
#                                                                                                      # data
#        # split the variables with various entries: 
#        one_hot = df.copy().loc[:, feature].str.split(', ', expand=True).stack()
#        
#        one_hot = pd.get_dummies(one_hot, prefix=feature, drop_first=True).groupby(level=0).sum()
#        df_tmp = df_tmp.drop(feature, axis=1)
#        df_tmp = df_tmp.merge(one_hot, left_index=True, right_index=True)
#    
#    return df_tmp